## Ovis2 34B

* From AIDC-AI
* Ranked #4 in OpenCompass multimodal academic leaderboard for open-source models (https://rank.opencompass.org.cn/leaderboard-multimodal)


In [1]:
# conda activate rca
'''
Ovis2-34B 
'''

import torch
from PIL import Image
from transformers import AutoModelForCausalLM

# load model
model_path = '/data/students/earl/llava-dissector/Ovis2-34B'
model = AutoModelForCausalLM.from_pretrained(model_path,
                                             torch_dtype=torch.bfloat16,
                                             multimodal_max_length=32768,
                                             trust_remote_code=True,
                                             device_map='auto') # distribute the load across multiple available GPUs
text_tokenizer = model.get_text_tokenizer()
visual_tokenizer = model.get_visual_tokenizer()

/data/students/earl/anaconda3/envs/rca/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.
Loading checkpoint shards:   0%|          | 0/17 [00:00<?, ?it/s]/data/students/earl/anaconda3/envs/rca/lib/python3.10/site-packages/torch/nn/modules/module.py:2400: UserWarning: for model.embed_tokens.weight: copying from a non-meta parameter in the checkpoint to a meta parameter in the current model, which is a no-op. (Did you mean to pass `assign=True` to assign items in the state dictionary to their corresponding key in the module instead of copying them in place?)
  warnings.warn(
/data/students/earl/anaconda3/envs/rca/lib/python3.10/site-packages/torch/nn/modules/module.py:2400: UserWarning: for model.layers.0

In [ ]:
import cv2
import io
import requests
import numpy as np
from PIL import Image

cls = "baseball helmet"

url = "https://farm3.staticflickr.com/2402/2480652763_e6b62303ee_z.jpg"
response = requests.get(url)
orig_img = Image.open(io.BytesIO(response.content)).convert('RGB')
orig_img_np = np.array(orig_img)  # HWC, RGB, uint8
img = orig_img_np #cv2.cvtColor(orig_img_np, cv2.COLOR_RGB2BGR)


text = f'Give the normalized bounding box coordinates in the format [x1, y1, x2, y2] of all instances of {cls} in the image'
#text = f'Give the bounding box coordinates of all instances of {cls} in the image.'
#text = f'Describe the image in detail.'
query = f'<image>\n{text}'
images = [orig_img]
max_partition = 9

model.threshold = -1.1

# format conversation
prompt, input_ids, pixel_values = model.preprocess_inputs(query, images, max_partition=max_partition)
attention_mask = torch.ne(input_ids, text_tokenizer.pad_token_id)
input_ids = input_ids.unsqueeze(0).to(device=model.device)
attention_mask = attention_mask.unsqueeze(0).to(device=model.device)
if pixel_values is not None:
    pixel_values = pixel_values.to(dtype=visual_tokenizer.dtype, device=visual_tokenizer.device)
pixel_values = [pixel_values]

# generate output
with torch.inference_mode():
    gen_kwargs = dict(
        max_new_tokens=1024,
        do_sample=False,
        top_p=None,
        top_k=None,
        temperature=None,
        repetition_penalty=None,
        eos_token_id=model.generation_config.eos_token_id,
        pad_token_id=text_tokenizer.pad_token_id,
        use_cache=True
    )
    output_ids = model.generate(
        input_ids, 
        pixel_values=pixel_values, 
        attention_mask=attention_mask, 
        **gen_kwargs
    )[0]
    output = text_tokenizer.decode(output_ids, skip_special_tokens=True)
    print(f'Output:\n{output}')
    
    
img_width, img_height = img.shape[1], img.shape[0]
print(f'Image Width: {img_width}, Image Height: {img_height}')

# Using cv2 to draw bounding boxes on the image
import re
import matplotlib.pyplot as plt

#pattern = r'\[\s*([\d.]+)\s*,\s*([\d.]+)\s*,\s*([\d.]+)\s*,\s*([\d.]+)\s*\]'
pattern = r'(?:[\[\(]\s*([\d.]+)\s*,\s*([\d.]+)\s*,\s*([\d.]+)\s*,\s*([\d.]+)\s*[\]\)])|(?:<box>\s*([\d.]+)\s+([\d.]+)\s+([\d.]+)\s+([\d.]+)\s*</box>)'
matches = re.findall(pattern, output)
bounding_boxes = [[float(coord) for coord in match] for match in matches]
print("Bounding boxes:", bounding_boxes)

# Visualize using cv2
import cv2
import numpy as np
img_cv = img #np.array(img)
for box in bounding_boxes:
    x1, y1, x2, y2 = box
    # convert these normalized coordinates to pixel values
    h, w, _ = img_cv.shape
    x1 = int(x1 * w)
    y1 = int(y1 * h)
    x2 = int(x2 * w)
    y2 = int(y2 * h)
    # Draw the bounding box and label on the image
    cv2.rectangle(img_cv, (x1, y1), (x2, y2), (0, 255, 0), 2)
    cv2.putText(img_cv, cls, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

plt.figure(figsize=(10, 10))
plt.imshow(img_cv)
plt.axis('off')
plt.show() 


RuntimeError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
